# Battery RUL Prediction - CatBoost GPU Training

This notebook trains a CatBoost regression model to predict Battery Remaining Useful Life (RUL) using Kaggle GPU (P100).

**Dataset**: Battery telemetry data in Parquet format
**Target**: RUL in days
**Model**: CatBoost with GPU acceleration

**Author**: Battery RUL Prediction System
**Version**: 1.0

## 1. Environment Setup

In [ ]:
# Install dependencies
!pip install -q catboost==1.2 pyarrow==15.0.0 pandas==2.1.4 scikit-learn matplotlib seaborn

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime, timedelta

# ML libraries
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully")
print(f"GPU Available: {CatBoostRegressor()._get_gpu_device_count() > 0}")

## 2. Data Loading from Parquet

In [ ]:
# Define paths (adjust based on Kaggle dataset structure)
DATA_DIR = Path('/kaggle/input/battery-rul-parquet')  # Update with actual Kaggle dataset name

# Alternative: if data is in working directory
if not DATA_DIR.exists():
    DATA_DIR = Path('/kaggle/working/data/parquet')

print(f"Data directory: {DATA_DIR}")
print(f"Directory exists: {DATA_DIR.exists()}")

In [ ]:
# Load master data
print("Loading master data...")
df_battery = pd.read_parquet(DATA_DIR / 'master' / 'battery.parquet')
df_location = pd.read_parquet(DATA_DIR / 'master' / 'location.parquet')

print(f"Batteries: {len(df_battery)}")
print(f"Locations: {len(df_location)}")

# Display sample
df_battery.head()

In [ ]:
# Load telemetry data
print("Loading telemetry data...")
df_raw_telemetry = pd.read_parquet(DATA_DIR / 'telemetry' / 'raw_telemetry.parquet')
df_calc_telemetry = pd.read_parquet(DATA_DIR / 'telemetry' / 'calc_telemetry.parquet')

print(f"Raw telemetry records: {len(df_raw_telemetry):,}")
print(f"Calculated telemetry records: {len(df_calc_telemetry):,}")

# Convert timestamps
df_raw_telemetry['ts'] = pd.to_datetime(df_raw_telemetry['ts'])
df_calc_telemetry['ts'] = pd.to_datetime(df_calc_telemetry['ts'])

print(f"\nDate range: {df_raw_telemetry['ts'].min()} to {df_raw_telemetry['ts'].max()}")
df_raw_telemetry.head()

In [ ]:
# Load RUL predictions (ground truth labels)
print("Loading RUL predictions...")
df_rul = pd.read_parquet(DATA_DIR / 'ml' / 'rul_predictions.parquet')
df_rul['prediction_time'] = pd.to_datetime(df_rul['prediction_time'])

print(f"RUL records: {len(df_rul):,}")
print(f"\nRUL statistics:")
print(df_rul['rul_days'].describe())

df_rul.head()

In [ ]:
# Load feature store (pre-aggregated features)
print("Loading feature store...")
df_features = pd.read_parquet(DATA_DIR / 'ml' / 'feature_store.parquet')
df_features['window_end'] = pd.to_datetime(df_features['window_end'])

print(f"Feature store records: {len(df_features):,}")
print(f"\nFeatures available: {df_features.columns.tolist()}")

df_features.head()

## 3. Feature Engineering

In [ ]:
# Merge features with RUL labels
print("Merging features with RUL labels...")

# Match by battery_id and nearest timestamp
df_train = pd.merge_asof(
    df_features.sort_values('window_end'),
    df_rul.sort_values('prediction_time'),
    left_on='window_end',
    right_on='prediction_time',
    by='battery_id',
    direction='nearest',
    tolerance=pd.Timedelta('1 hour')
)

# Remove rows without RUL labels
df_train = df_train.dropna(subset=['rul_days'])

print(f"Training samples after merge: {len(df_train):,}")
print(f"Batteries in training set: {df_train['battery_id'].nunique()}")

In [ ]:
# Define feature columns
voltage_features = ['v_mean', 'v_std', 'v_min', 'v_max', 'v_range']
temperature_features = ['t_mean', 't_std', 't_min', 't_max', 't_delta_from_ambient']
resistance_features = ['r_internal_latest', 'r_internal_trend']
operational_features = ['discharge_cycles_count', 'ah_throughput', 'time_at_high_temp_pct']

# Combine all features
feature_cols = (
    voltage_features + 
    temperature_features + 
    resistance_features + 
    operational_features
)

# Verify all features exist
missing_features = [f for f in feature_cols if f not in df_train.columns]
if missing_features:
    print(f"Warning: Missing features: {missing_features}")
    feature_cols = [f for f in feature_cols if f in df_train.columns]

print(f"\nFeatures selected for training ({len(feature_cols)}):")
for i, feat in enumerate(feature_cols, 1):
    print(f"  {i}. {feat}")

In [ ]:
# Create derived features
print("Creating derived features...")

# Voltage health indicator
df_train['v_health_score'] = (
    (df_train['v_mean'] - 11.5) / (13.65 - 11.5)  # Normalize to 0-1 range
).clip(0, 1)

# Temperature stress indicator
df_train['t_stress_score'] = (
    (df_train['t_max'] - 25) / 20  # Higher temp = more stress
).clip(0, 1)

# Resistance degradation rate
if 'r_internal_latest' in df_train.columns and 'r_internal_trend' in df_train.columns:
    df_train['r_degradation_rate'] = df_train['r_internal_trend'] / df_train['r_internal_latest']
    feature_cols.append('r_degradation_rate')

# Usage intensity
if 'ah_throughput' in df_train.columns and 'discharge_cycles_count' in df_train.columns:
    df_train['usage_intensity'] = df_train['ah_throughput'] / (df_train['discharge_cycles_count'] + 1)
    feature_cols.append('usage_intensity')

# Add derived features
feature_cols.extend(['v_health_score', 't_stress_score'])

print(f"Total features after engineering: {len(feature_cols)}")

In [ ]:
# Handle missing values
print("Handling missing values...")
print(f"\nMissing values per feature:")
missing_counts = df_train[feature_cols].isnull().sum()
print(missing_counts[missing_counts > 0])

# Fill missing values with median
for col in feature_cols:
    if df_train[col].isnull().any():
        df_train[col].fillna(df_train[col].median(), inplace=True)

print(f"\nData shape after cleaning: {df_train.shape}")

## 4. Exploratory Data Analysis

In [ ]:
# RUL distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df_train['rul_days'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('RUL (days)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('RUL Distribution')
axes[0].axvline(df_train['rul_days'].median(), color='red', linestyle='--', label=f'Median: {df_train["rul_days"].median():.1f}')
axes[0].legend()

axes[1].boxplot(df_train['rul_days'])
axes[1].set_ylabel('RUL (days)')
axes[1].set_title('RUL Box Plot')

plt.tight_layout()
plt.savefig('/kaggle/working/rul_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"RUL Statistics:")
print(df_train['rul_days'].describe())

In [ ]:
# Feature correlations with RUL
correlations = df_train[feature_cols + ['rul_days']].corr()['rul_days'].drop('rul_days').sort_values()

plt.figure(figsize=(10, 8))
correlations.plot(kind='barh', color=['red' if x < 0 else 'green' for x in correlations])
plt.xlabel('Correlation with RUL')
plt.title('Feature Correlations with RUL')
plt.tight_layout()
plt.savefig('/kaggle/working/feature_correlations.png', dpi=150, bbox_inches='tight')
plt.show()

print("Top 10 features correlated with RUL:")
print(correlations.abs().sort_values(ascending=False).head(10))

In [ ]:
# Key features scatter plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

key_features = ['v_mean', 't_max', 'r_internal_latest', 'discharge_cycles_count']
for idx, feat in enumerate(key_features):
    if feat in df_train.columns:
        row, col = idx // 2, idx % 2
        axes[row, col].scatter(df_train[feat], df_train['rul_days'], alpha=0.5, s=10)
        axes[row, col].set_xlabel(feat)
        axes[row, col].set_ylabel('RUL (days)')
        axes[row, col].set_title(f'RUL vs {feat}')

plt.tight_layout()
plt.savefig('/kaggle/working/rul_scatter_plots.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Train/Test Split

In [ ]:
# Prepare features and target
X = df_train[feature_cols].copy()
y = df_train['rul_days'].copy()

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"\nFeature value ranges:")
print(X.describe())

In [ ]:
# Stratified split by RUL bins to ensure balanced distribution
rul_bins = pd.cut(y, bins=5, labels=['very_low', 'low', 'medium', 'high', 'very_high'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=rul_bins
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"\nRUL distribution in training set:")
print(y_train.describe())
print(f"\nRUL distribution in test set:")
print(y_test.describe())

## 6. CatBoost GPU Training

In [ ]:
# Create CatBoost pools
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

print("CatBoost pools created successfully")

In [ ]:
# Configure CatBoost model with GPU
model = CatBoostRegressor(
    # GPU Configuration
    task_type='GPU',
    devices='0',
    
    # Model hyperparameters
    iterations=2000,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3,
    
    # Loss function
    loss_function='RMSE',
    eval_metric='MAE',
    
    # Regularization
    random_strength=1,
    bagging_temperature=1,
    
    # Early stopping
    early_stopping_rounds=100,
    use_best_model=True,
    
    # Output
    verbose=100,
    random_seed=42
)

print("CatBoost model configured:")
print(f"  Task type: GPU")
print(f"  Iterations: 2000")
print(f"  Learning rate: 0.05")
print(f"  Tree depth: 8")

In [ ]:
# Train model
print("Starting training...\n")
start_time = datetime.now()

model.fit(
    train_pool,
    eval_set=test_pool,
    plot=True
)

training_time = (datetime.now() - start_time).total_seconds()
print(f"\nTraining completed in {training_time:.1f} seconds ({training_time/60:.1f} minutes)")
print(f"Best iteration: {model.get_best_iteration()}")

## 7. Model Evaluation

In [ ]:
# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calculate metrics
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

# Display results
print("="*60)
print("MODEL PERFORMANCE")
print("="*60)
print(f"\nTraining Set:")
print(f"  MAE:  {train_mae:.2f} days")
print(f"  RMSE: {train_rmse:.2f} days")
print(f"  R²:   {train_r2:.4f}")

print(f"\nTest Set:")
print(f"  MAE:  {test_mae:.2f} days")
print(f"  RMSE: {test_rmse:.2f} days")
print(f"  R²:   {test_r2:.4f}")

print(f"\nOverfitting Check:")
print(f"  MAE gap:  {abs(test_mae - train_mae):.2f} days")
print(f"  RMSE gap: {abs(test_rmse - train_rmse):.2f} days")
print("="*60)

In [ ]:
# Prediction accuracy analysis
test_errors = np.abs(y_test - y_pred_test)

print("\nPrediction Error Analysis:")
print(f"  Mean error: {test_errors.mean():.2f} days")
print(f"  Median error: {test_errors.median():.2f} days")
print(f"  90th percentile error: {test_errors.quantile(0.9):.2f} days")
print(f"  Max error: {test_errors.max():.2f} days")

# Accuracy within thresholds
within_7_days = (test_errors <= 7).mean() * 100
within_30_days = (test_errors <= 30).mean() * 100
within_60_days = (test_errors <= 60).mean() * 100

print(f"\nPrediction Accuracy:")
print(f"  Within 7 days:  {within_7_days:.1f}%")
print(f"  Within 30 days: {within_30_days:.1f}%")
print(f"  Within 60 days: {within_60_days:.1f}%")

In [ ]:
# Visualization: Predicted vs Actual
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
axes[0].scatter(y_test, y_pred_test, alpha=0.5, s=20)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect prediction')
axes[0].set_xlabel('Actual RUL (days)')
axes[0].set_ylabel('Predicted RUL (days)')
axes[0].set_title(f'Predicted vs Actual RUL (Test Set)\nMAE: {test_mae:.2f} days, R²: {test_r2:.4f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals
residuals = y_test - y_pred_test
axes[1].scatter(y_pred_test, residuals, alpha=0.5, s=20)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted RUL (days)')
axes[1].set_ylabel('Residuals (days)')
axes[1].set_title('Residual Plot')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/kaggle/working/prediction_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Error distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(test_errors, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Absolute Error (days)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Prediction Errors')
axes[0].axvline(test_mae, color='red', linestyle='--', label=f'MAE: {test_mae:.2f}')
axes[0].legend()

axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Residual (days)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Residuals')
axes[1].axvline(0, color='red', linestyle='--', label='Zero error')
axes[1].legend()

plt.tight_layout()
plt.savefig('/kaggle/working/error_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Feature Importance Analysis

In [ ]:
# Get feature importance
feature_importance = model.get_feature_importance()
feature_names = X_train.columns

# Create DataFrame
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("Feature Importance (Top 15):")
print(importance_df.head(15))

# Save to CSV
importance_df.to_csv('/kaggle/working/feature_importance.csv', index=False)
print("\nFeature importance saved to feature_importance.csv")

In [ ]:
# Visualize feature importance
plt.figure(figsize=(10, 8))
top_n = 20
top_features = importance_df.head(top_n)

plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance')
plt.title(f'Top {top_n} Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('/kaggle/working/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Model Export

In [ ]:
# Save model in multiple formats
model_dir = Path('/kaggle/working')

# 1. Native CatBoost format (.cbm)
model_path_cbm = model_dir / 'rul_model.cbm'
model.save_model(str(model_path_cbm))
print(f"Model saved (CatBoost): {model_path_cbm}")

# 2. ONNX format for deployment
try:
    model_path_onnx = model_dir / 'rul_model.onnx'
    model.save_model(
        str(model_path_onnx),
        format="onnx",
        export_parameters={
            'onnx_domain': 'ai.catboost',
            'onnx_model_version': 1,
            'onnx_doc_string': 'Battery RUL Prediction Model'
        }
    )
    print(f"Model saved (ONNX): {model_path_onnx}")
except Exception as e:
    print(f"ONNX export failed: {e}")

In [ ]:
# Save model metadata
metadata = {
    'model_type': 'CatBoostRegressor',
    'task': 'Battery RUL Prediction',
    'target': 'rul_days',
    'training_date': datetime.now().isoformat(),
    'training_time_seconds': training_time,
    
    # Data info
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'features': feature_cols,
    'num_features': len(feature_cols),
    
    # Hyperparameters
    'hyperparameters': {
        'iterations': model.get_param('iterations'),
        'learning_rate': model.get_param('learning_rate'),
        'depth': model.get_param('depth'),
        'l2_leaf_reg': model.get_param('l2_leaf_reg'),
    },
    
    # Performance metrics
    'metrics': {
        'train': {
            'mae': float(train_mae),
            'rmse': float(train_rmse),
            'r2': float(train_r2)
        },
        'test': {
            'mae': float(test_mae),
            'rmse': float(test_rmse),
            'r2': float(test_r2)
        },
        'accuracy_thresholds': {
            'within_7_days_pct': float(within_7_days),
            'within_30_days_pct': float(within_30_days),
            'within_60_days_pct': float(within_60_days)
        }
    },
    
    # Feature importance
    'top_10_features': importance_df.head(10).to_dict('records')
}

# Save metadata
metadata_path = model_dir / 'model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\nModel metadata saved: {metadata_path}")
print(f"\nMetadata summary:")
print(f"  Features: {metadata['num_features']}")
print(f"  Training samples: {metadata['training_samples']:,}")
print(f"  Test MAE: {metadata['metrics']['test']['mae']:.2f} days")
print(f"  Test R²: {metadata['metrics']['test']['r2']:.4f}")

In [ ]:
# Create deployment package
import zipfile

deployment_package = model_dir / 'rul_model_deployment.zip'

with zipfile.ZipFile(deployment_package, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add model files
    zipf.write(model_path_cbm, 'rul_model.cbm')
    if (model_dir / 'rul_model.onnx').exists():
        zipf.write(model_dir / 'rul_model.onnx', 'rul_model.onnx')
    
    # Add metadata and documentation
    zipf.write(metadata_path, 'model_metadata.json')
    zipf.write(model_dir / 'feature_importance.csv', 'feature_importance.csv')
    
    # Add visualizations
    for viz in ['rul_distribution.png', 'feature_correlations.png', 
                'prediction_analysis.png', 'feature_importance.png', 'error_distribution.png']:
        viz_path = model_dir / viz
        if viz_path.exists():
            zipf.write(viz_path, viz)

print(f"\nDeployment package created: {deployment_package}")
print(f"Package size: {deployment_package.stat().st_size / 1024 / 1024:.2f} MB")

## 10. Summary Report

In [ ]:
# Generate training report
report = f"""
{'='*80}
BATTERY RUL PREDICTION MODEL - TRAINING REPORT
{'='*80}

Training Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Training Duration: {training_time:.1f} seconds ({training_time/60:.1f} minutes)

DATASET INFORMATION
-------------------
Training samples: {len(X_train):,}
Test samples: {len(X_test):,}
Total features: {len(feature_cols)}
RUL range: {y.min():.1f} - {y.max():.1f} days
RUL mean: {y.mean():.1f} days
RUL std: {y.std():.1f} days

MODEL CONFIGURATION
-------------------
Algorithm: CatBoostRegressor (GPU)
Iterations: {model.get_param('iterations')}
Learning rate: {model.get_param('learning_rate')}
Tree depth: {model.get_param('depth')}
L2 regularization: {model.get_param('l2_leaf_reg')}
Best iteration: {model.get_best_iteration()}

PERFORMANCE METRICS
-------------------
Training Set:
  MAE:  {train_mae:.2f} days
  RMSE: {train_rmse:.2f} days
  R²:   {train_r2:.4f}

Test Set:
  MAE:  {test_mae:.2f} days
  RMSE: {test_rmse:.2f} days
  R²:   {test_r2:.4f}

Prediction Accuracy:
  Within 7 days:  {within_7_days:.1f}%
  Within 30 days: {within_30_days:.1f}%
  Within 60 days: {within_60_days:.1f}%

TOP 10 IMPORTANT FEATURES
-------------------------
"""

for idx, row in importance_df.head(10).iterrows():
    report += f"{row['feature']:30s} {row['importance']:8.2f}\n"

report += f"""
OUTPUT FILES
------------
✓ rul_model.cbm (CatBoost format)
✓ rul_model.onnx (ONNX format - if available)
✓ model_metadata.json (Complete model information)
✓ feature_importance.csv (Feature rankings)
✓ rul_model_deployment.zip (Deployment package)
✓ Visualization plots (PNG)

DEPLOYMENT INSTRUCTIONS
-----------------------
1. Download outputs:
   kaggle kernels output khiwnitithadachot/kaggle-notebook-optimized -p ./model

2. Load model in production:
   from catboost import CatBoostRegressor
   model = CatBoostRegressor()
   model.load_model('rul_model.cbm')

3. Make predictions:
   predictions = model.predict(X)

{'='*80}
TRAINING COMPLETED SUCCESSFULLY
{'='*80}
"""

print(report)

# Save report
report_path = model_dir / 'TRAINING_REPORT.txt'
with open(report_path, 'w') as f:
    f.write(report)

print(f"\nReport saved: {report_path}")

In [ ]:
# List all output files
print("\nOutput files available for download:")
print("="*60)
for file in sorted(model_dir.glob('*')):
    if file.is_file() and not file.name.startswith('.'):
        size_mb = file.stat().st_size / 1024 / 1024
        print(f"  {file.name:40s} {size_mb:8.2f} MB")
print("="*60)